In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

# --- 1. Load and Prepare Data ---
try:
    df = pd.read_csv("arecanut.csv")
except FileNotFoundError:
    print("Error: 'arecanut.csv' not found. Please make sure the CSV file is in the same directory.")
    exit()

df['Price Date'] = pd.to_datetime(df['Price Date'])
df.sort_values('Price Date', inplace=True)
price_data = df[['Modal Price (Rs./Quintal)']].values

# --- 2. Scale the Data ---
# LSTMs work best with data scaled between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(price_data)

# --- 3. Create Training Sequences ---
# We'll use the last 30 days of data to predict the next day's price
SEQUENCE_LENGTH = 30
X, y = [], []
for i in range(len(scaled_data) - SEQUENCE_LENGTH):
    X.append(scaled_data[i:(i + SEQUENCE_LENGTH), 0])
    y.append(scaled_data[i + SEQUENCE_LENGTH, 0])

X, y = np.array(X), np.array(y)

# Reshape X to be [samples, timesteps, features] which is required for LSTM
X = np.reshape(X, (X.shape[0], X.shape[1], 1))


# --- 4. Build and Train the LSTM Model ---
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, batch_size=32, verbose=1)

# --- 5. Save the Model and Scaler ---
model.save('model_patora.h5')
joblib.dump(scaler, 'scaler_patora.gz')

print("\\nTraining complete. Model saved as 'model_patora.h5' and scaler as 'scaler_patora.gz'.")

Epoch 1/100
102/102 [==============================] - 2s 10ms/step - loss: 0.0144
Epoch 2/100
102/102 [==============================] - 1s 10ms/step - loss: 0.0052
Epoch 3/100
102/102 [==============================] - 1s 10ms/step - loss: 0.0051
Epoch 4/100
102/102 [==============================] - 1s 10ms/step - loss: 0.0053
Epoch 5/100
102/102 [==============================] - 1s 10ms/step - loss: 0.0053
Epoch 6/100
102/102 [==============================] - 1s 10ms/step - loss: 0.0052
Epoch 7/100
102/102 [==============================] - 1s 10ms/step - loss: 0.0052
Epoch 8/100
102/102 [==============================] - 1s 10ms/step - loss: 0.0051
Epoch 9/100
102/102 [==============================] - 1s 11ms/step - loss: 0.0051
Epoch 10/100
102/102 [==============================] - 1s 11ms/step - loss: 0.0051
Epoch 11/100
102/102 [==============================] - 1s 11ms/step - loss: 0.0051
Epoch 12/100
102/102 [==============================] - 1s 10ms/step - loss: 0.0051
E

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
